In [ ]:
import pathlib
import time
from importlib import reload
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import cf_xarray as cfxr

In [4]:
from numba import njit, guvectorize, prange
import dask.config
dask.config.set(scheduler='threads')

In [10]:
@guvectorize(["void(boolean[:], boolean[:], uint16[:])"], "(n),(n)->()", nopython=True)
def eca_precursor(b1, b2wr, KRprec):
    KRprec[0] = np.sum(b1 & b2wr)

In [6]:
@guvectorize(["void(bool_[:,:], bool_[:,:], uint16[:,:])"], "(m,n),(l,n)->(m,l)", nopython=True,)
def eca_precursors_pair(b2, b1w, result):
    """一次性计算所有位置对的 precursor 关系"""
    m = b2.shape[0]     # 第一数组的第一维
    l = b1w.shape[0]    # 第二数组的第一维
    
    for i in range(m):
        for j in range(l):
            result[i, j] = np.sum(b2[i, :] & b1w[j, :])

In [11]:
@njit(cache=True)
def eca_precursor_njit(b1, b2wr):
    KRprec = np.sum(b1 & b2wr)
    return KRprec

In [8]:
@njit(cache=True)
def eca_precursors_pair_njit(b2, b1w):
    """一次性计算所有位置对的 precursor 关系"""
    result = np.zeros((b2.shape[0], b1w.shape[0]), dtype=np.uint16)
    m = b2.shape[0]     # 第一数组的第一维
    l = b1w.shape[0]    # 第二数组的第一维
    
    for i in range(m):
        for j in range(l):
            result[i, j] = np.sum(b2[i, :] & b1w[j, :])
    return result

In [65]:
@njit(cache=True, parallel=True)
def eca_precursors_pair_njitp(b2, b1w):
    """一次性计算所有位置对的 precursor 关系"""
    result = np.zeros((b2.shape[0], b1w.shape[0]), dtype=np.uint16)
    m = b2.shape[0]     # 第一数组的第一维
    l = b1w.shape[0]    # 第二数组的第一维
    
    for i in prange(m):
        for j in range(l):
            result[i, j] = np.sum(b2[i, :] & b1w[j, :])
    return result

## prepare data

In [36]:
ds = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .isel(lat=slice(0, 100), lon=slice(0, 20))
ds

<xarray.Dataset> Size: 88MB
Dimensions:  (time: 10957, lon: 20, lat: 100)
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lon      (lon) float64 160B 73.0 73.25 73.5 73.75 ... 77.0 77.25 77.5 77.75
  * lat      (lat) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
Data variables:
    SPI1     (time, lat, lon) float32 88MB ...
Attributes:
    CDI:          Climate Data Interface version 2.4.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Thu Mar 06 20:30:32 2025: cdo -L selindexbox,293,542,144,34...
    CDO:          Climate Data Operators version 2.4.4 (https://mpimet.mpg.de...

In [37]:
da_droughtA = (ds["SPI1"] < -1).copy().rename({"lon": "lonA", "lat": "latA"})
da_droughtB = (ds["SPI1"] < -1).copy().rename({"lon": "lonB", "lat": "latB"})

In [38]:
da_droughtA_stack = da_droughtA.stack({"locA": ["latA", "lonA"]}).T
da_droughtB_stack = da_droughtB.stack({"locB": ["latB", "lonB"]}).T

### `njit` cases

In [40]:
%%timeit -n 3 -r 3
xr.apply_ufunc(eca_precursor_njit, da_droughtA, da_droughtB, vectorize=True,
               input_core_dims=[["time"], ["time"]], output_core_dims=[[]], 
               ) #dask="parallelized", output_dtypes=[np.uint16] 

3min 34s ± 272 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [41]:
%%timeit -n 1 -r 1
xr.apply_ufunc(eca_precursors_pair_njit, da_droughtA, da_droughtB, vectorize=True, 
               input_core_dims=[["lonA", "time"], ["lonB", "time"]], 
               output_core_dims=[["lonA", "lonB"]],
               )

3min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
tic = time.time()
eca_precursors_pair_njit(da_droughtA_stack.values, da_droughtB_stack.values)
print(time.time() - tic)

1741330015.1323948
1741330220.0367894


### `guvectorize`

In [42]:
%%timeit -n 1 -r 1
xr.apply_ufunc(eca_precursor, da_droughtA, da_droughtB, vectorize=False,
                input_core_dims=[["time"], ["time"]], output_core_dims=[[]], 
                # dask="parallelized", output_dtypes=[np.uint16]
                )

3min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [43]:
%%timeit -n 1 -r 1
xr.apply_ufunc(eca_precursors_pair, da_droughtA, da_droughtB, vectorize=False,
                input_core_dims=[["lonA", "time"], ["lonB", "time"]], 
                output_core_dims=[["lonA", "lonB"]],
                # dask="parallelized"
                )

3min 5s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
%%timeit -n 1 -r 1
xr.apply_ufunc(eca_precursors_pair, da_droughtA, da_droughtB, vectorize=False,
                input_core_dims=[["latA", "time"], ["latB", "time"]], 
                output_core_dims=[["latA", "latB"]],
                # dask="parallelized"
                )

5min 56s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Dask

In [86]:
ds_chunk = xr.open_dataset("tests/data/era5.reanalysis.spi30d.0p25deg.china.1950-1979.nc").rename({"latitude": "lat", "longitude": "lon", "spi30d": "SPI1"})\
        .isel(lat=slice(0, 100), lon=slice(0, 100)).chunk({"lat": 20, "lon": 20})
da_droughtA_chunk = (ds_chunk["SPI1"] < -1).copy().rename({"lon": "lonA", "lat": "latA"})
da_droughtB_chunk = (ds_chunk["SPI1"] < -1).copy().rename({"lon": "lonB", "lat": "latB"})

In [60]:
da_droughtA_chunk

<xarray.DataArray 'SPI1' (time: 10957, latA: 100, lonA: 100)> Size: 110MB
dask.array<lt, shape=(10957, 100, 100), dtype=bool, chunksize=(10957, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 88kB 1950-01-01 1950-01-02 ... 1979-12-31
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5

In [87]:
# %%timeit -n 1 -r 1
xr.apply_ufunc(eca_precursor, da_droughtA_chunk, da_droughtB_chunk, vectorize=False,
               input_core_dims=[["time"], ["time"]], output_core_dims=[[]],  
               dask="parallelized",
               ).compute()

/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(


<xarray.DataArray 'SPI1' (latA: 100, lonA: 100, latB: 100, lonB: 100)> Size: 200MB
array([[[[1822, 1723, 1646, ...,  392,  356,  365],
         [1668, 1628, 1590, ...,  426,  365,  353],
         [1547, 1531, 1542, ...,  521,  470,  400],
         ...,
         [ 322,  341,  356, ...,  311,  258,  230],
         [ 306,  340,  327, ...,  284,  292,  232],
         [ 296,  303,  306, ...,  278,  324,  255]],

        [[1723, 1808, 1704, ...,  411,  374,  381],
         [1629, 1661, 1634, ...,  438,  378,  370],
         [1550, 1550, 1571, ...,  532,  480,  404],
         ...,
         [ 309,  328,  343, ...,  304,  250,  222],
         [ 292,  329,  320, ...,  268,  284,  227],
         [ 293,  298,  303, ...,  260,  314,  246]],

        [[1646, 1704, 1805, ...,  416,  384,  387],
         [1611, 1653, 1668, ...,  448,  393,  384],
         [1560, 1563, 1575, ...,  539,  492,  423],
         ...,
...
         ...,
         [ 238,  235,  226, ..., 1076, 1059, 1064],
         [ 236,  260,  257, ..., 1474, 1163, 1118],
         [ 228,  240,  254, ..., 1878, 1373, 1202]],

        [[ 324,  314,  310, ...,  304,  291,  301],
         [ 299,  295,  297, ...,  253,  272,  293],
         [ 324,  318,  314, ...,  253,  278,  294],
         ...,
         [ 174,  174,  191, ..., 1359, 1312, 1222],
         [ 171,  190,  212, ..., 1393, 1581, 1324],
         [ 165,  186,  202, ..., 1373, 1919, 1448]],

        [[ 255,  246,  238, ...,  255,  246,  245],
         [ 231,  234,  236, ...,  195,  222,  244],
         [ 252,  250,  241, ...,  202,  223,  241],
         ...,
         [ 164,  163,  178, ..., 1173, 1202, 1202],
         [ 163,  188,  208, ..., 1229, 1343, 1400],
         [ 155,  180,  197, ..., 1202, 1448, 1664]]]], dtype=uint16)
Coordinates:
  * lonA     (lonA) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latA     (latA) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5
  * lonB     (lonB) float64 800B 73.0 73.25 73.5 73.75 ... 97.0 97.25 97.5 97.75
  * latB     (latB) float64 800B 54.25 54.0 53.75 53.5 ... 30.25 30.0 29.75 29.5

In [63]:
%%timeit -n 3 -r 3
xr.apply_ufunc(eca_precursors_pair, da_droughtA_chunk, da_droughtB_chunk, vectorize=False,
                input_core_dims=[["lonA", "time"], ["lonB", "time"]], 
                output_core_dims=[["lonA", "lonB"]],
                dask="parallelized", dask_gufunc_kwargs={"allow_rechunk": True}
                ).compute()

/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(


/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarning: Increasing number of chunks by factor of 25
  tmp = blockwise(
/home/hmwang-cli/.conda/envs/syncfuture/lib/python3.11/site-packages/dask/array/gufunc.py:458: PerformanceWarn

1min 4s ± 800 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [ ]:
da_droughtA_unchunk_stack = da_droughtA_chunk.stack({"locA": ["latA", "lonA"]}).T.values
da_droughtB_unchunk_stack = da_droughtB_chunk.stack({"locB": ["latB", "lonB"]}).T.values

In [70]:
eca_precursors_pair_njitp(da_droughtA_unchunk_stack, da_droughtB_unchunk_stack)

array([[1822, 1723, 1646, ...,  278,  324,  255],
       [1723, 1808, 1704, ...,  260,  314,  246],
       [1646, 1704, 1805, ...,  254,  310,  238],
       ...,
       [ 278,  260,  254, ..., 1878, 1373, 1202],
       [ 324,  314,  310, ..., 1373, 1919, 1448],
       [ 255,  246,  238, ..., 1202, 1448, 1664]], dtype=uint16)

41.9s vs. 48.4s